In [52]:
import collections
import functools
import json
import time

from googletrans import Translator; translator = Translator()
from tqdm.notebook import tqdm

from pan20 import fake
from pan20.util import chis, text

In [2]:
en = fake.load('en')
es = fake.load('es')

In [20]:
en['toks'] = en.tweet.apply(text.tokenize)
tokenize_es = functools.partial(text.tokenize, lang='es')
es['toks'] = es.tweet.apply(tokenize_es)

In [48]:
def get_chis(df):
    c0 = collections.Counter()
    c1 = collections.Counter()
    for _, x in df.iterrows():
        if x.label:
            c1.update(x.toks)
        else:
            c0.update(x.toks)
    return chis.get_chis(c1, c0)

en_chi = get_chis(en)
es_chi = get_chis(es)
en_chi = en_chi[en_chi.c1 > en_chi.c2]
es_chi = es_chi[es_chi.c1 > es_chi.c2]

In [62]:
es_chi[es_chi.token == '']

,token,c1,c2,n1,n2,chi


In [54]:
def translate(x, src, dest):
    try:
        time.sleep(0.2)
        return translator.translate(x, src=src, dest=dest).text
    except json.JSONDecodeError:
        print('Dang. Sleeping...')
        time.sleep(30)
    
en_to_es = functools.partial(translate, src='en', dest='es')
es_to_en = functools.partial(translate, src='es', dest='en')

In [55]:
es_from_en = []
with tqdm(total=len(en_chi)) as pbar:
    for _, x in en_chi.iterrows():
        es_ = en_to_es(x.token)
        es_from_en.append(es_)
        pbar.update()

Dang. Sleeping...
Dang. Sleeping...
Dang. Sleeping...
Dang. Sleeping...
Dang. Sleeping...
Dang. Sleeping...
Dang. Sleeping...



KeyboardInterrupt: 

In [50]:
en_chi['token_es'] = en_chi.token.apply(en_to_es)

KeyboardInterrupt: 

In [ ]:
es_chi['token_en'] = es_chi.token.apply(es_to_en)